In [ ]:
import whisper
import os
import json


print(f"Whisper version: {whisper.__version__}")



In [ ]:
# Convert .m4a or .mp4 to mp3
def convert_to_MP3(in_file, out_file):
    fileToConvert = AudioFileClip(in_file)
    fileToConvert.write_audiofile(out_file)
    fileToConvert.close()

IN_FILE_PATH = os.path.join(folder_path, "sound.mp4") # sound.m4a
OUT_FILE_PATH = os.path.join(folder_path, "sound.mp3") # sound.mp3

convert_to_MP3(IN_FILE_PATH, OUT_FILE_PATH)


In [ ]:
# Transkribera ljudet

audio_file_name = "ljud.wma"
audio_file_path = os.path.join(folder_path, audio_file_name)

language = "sv"
beamSize = 5
bestOf = 2

# Ladda ner modellen
model_name = r"large-v2"
model = whisper.load_model(model_name)

#result = model.transcribe(audio_file_path, language=language, beam_size=beamSize, best_of=bestOf) # Tar lång tid... 
result = model.transcribe(audio_file_path, language="sv")
#result = model.transcribe(audio_file_path, **transcribe_options) # Ny approach 
  
print(result["text"])

In [ ]:
#file_name = f"whisper_{whisper.__version__}_{model_name}_beamSize_{beamSize}_bestOf_{bestOf}"
file_name = f"default_whisper_{whisper.__version__}_{model_name}"


# Spara resultat till .json (har en massa extra information)
with open(os.path.join(folder_path, f"{file_name}.json"), "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

# Spara all text som .txt
with open(os.path.join(folder_path, f"{file_name}.txt"), "w", encoding="utf-8") as f:
    f.write(result["text"])

# Spara all text med radbryten som .txt
lines = []
for line in result["segments"]:
    lines.append(line["text"])
with open(os.path.join(folder_path, f"{file_name}_radbryte.txt"), "w", encoding="utf-8") as f:
    for line in lines:
        f.write(f"{line}\n")

In [ ]:
# .tsv-fil
with open(os.path.join(folder_path, f"{file_name}.tsv"), "w", encoding="utf-8") as f:

    print("start", "end", "text", sep="\t", file=f)
    for segment in result["segments"]:
        print(round(1000 * segment['start']), file=f, end="\t")
        print(round(1000 * segment['end']), file=f, end="\t")
        print(segment['text'].strip().replace("\t", " "), file=f, flush=True)

In [ ]:
# .srt-fil

def format_timestamp(seconds: float, always_include_hours: bool = False, decimal_marker: str = '.'):
    assert seconds >= 0, "non-negative timestamp expected"
    milliseconds = round(seconds * 1000.0)

    hours = milliseconds // 3_600_000
    milliseconds -= hours * 3_600_000

    minutes = milliseconds // 60_000
    milliseconds -= minutes * 60_000

    seconds = milliseconds // 1_000
    milliseconds -= seconds * 1_000

    hours_marker = f"{hours:02d}:" if always_include_hours or hours > 0 else ""
    return f"{hours_marker}{minutes:02d}:{seconds:02d}{decimal_marker}{milliseconds:03d}"

with open(os.path.join(folder_path, f"{file_name}.srt"), "w", encoding="utf-8") as f:
    for i, segment in enumerate(result["segments"], start=1):
        # write srt lines
        print(
            f"{i}\n"
            f"{format_timestamp(segment['start'], always_include_hours=True, decimal_marker='.')} --> "
            f"{format_timestamp(segment['end'], always_include_hours=True, decimal_marker='.')}\n"
            f"{segment['text'].strip().replace('-->', '->')}\n",
            file=f,
            flush=True,
        )

In [ ]:
# .vtt-fil
with open(os.path.join(folder_path, f"{file_name}.vtt"), "w", encoding="utf-8") as f:
    for segment in result["segments"]:
        print(
            f"{format_timestamp(segment['start'])} --> {format_timestamp(segment['end'])}\n"
            f"{segment['text'].strip().replace('-->', '->')}\n",
            file=f,
            flush=True,
        )